In [3]:
import iris 
import matplotlib.pyplot as plt
import iris.quickplot as qplt
import iris.plot as iplt
import numpy as np

%matplotlib inline

ModuleNotFoundError: No module named 'iris'

### Import simulations from data_dir and perturbed controls from data_dir1

In [2]:
data_dir = '/shared/netscratch/lrm49/UKCA_VN112_QBO_SIMS/JAN_E/'
data_dir1 = '/shared//netscratch/lrm49/UKCA_VN112_QBO_SIMS/PERTLIM_CONTROLS/'

In [4]:
cont= iris.load(data_dir+'u-bl775_all_vars_processed.nc')
a = iris.load(data_dir+'u-bo332_all_vars_processed.nc')
b = iris.load(data_dir+'u-bo334_all_vars_processed.nc')
c = iris.load(data_dir+'u-bo335_all_vars_processed.nc')
d = iris.load(data_dir+'u-bo336_all_vars_processed.nc')
e = iris.load(data_dir+'u-bo337_all_vars_processed.nc')
f = iris.load(data_dir+'u-bo338_all_vars_processed.nc')
JNE1 = iris.load(data_dir1+'u-br301_all_vars_processed.nc')
JNE2 = iris.load(data_dir1+'u-br302_all_vars_processed.nc')

# Constrain data to get equatorial x wind

### Select x wind

In [6]:
list_x = [a[11],  b[11], c[11], d[11], e[11], f[11]]

In [7]:
control_list = [cont[10], JNE1[11],JNE2[11]]

### Collapse over latitude

In [8]:
latlist = []
for eruption in list_x:
    lat_constraint = eruption.extract(iris.Constraint(latitude = lambda cell: -2 <= cell <=2))
    latlist.append(lat_constraint)

In [9]:
latlist_cont = []
for eruption in control_list:
    lat_constraint = eruption.extract(iris.Constraint(latitude = lambda cell: -2 <= cell <=2))
    latlist_cont.append(lat_constraint)

### Guess bounds for grid areas

In [10]:
import iris.analysis.cartography

In [11]:
latlist[0].coord('latitude').guess_bounds()
latlist[0].coord('longitude').guess_bounds()

In [12]:
grid_areas = iris.analysis.cartography.area_weights(latlist[0])

### Average over area

In [13]:
areaav_list = []
for eruption in latlist:
    area_average = eruption.collapsed(['latitude', 'longitude'], iris.analysis.MEAN, weights = grid_areas)
    areaav_list.append(area_average)

/usr/local/lib/python2.7/dist-packages/Iris-1.10.0-py2.7.egg/iris/coords.py:992: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for u'latitude'.
  warnings.warn(msg.format(self.name()))
/usr/local/lib/python2.7/dist-packages/Iris-1.10.0-py2.7.egg/iris/coords.py:992: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for u'longitude'.
  warnings.warn(msg.format(self.name()))


In [14]:
areaav_list_cont = []
for eruption in latlist_cont:
    area_average = eruption.collapsed(['latitude', 'longitude'], iris.analysis.MEAN, weights = grid_areas)
    areaav_list_cont.append(area_average)

In [ ]:
control_sum = areaav_list_cont[0] + areaav_list_cont[1] + areaav_list_cont[2]
control_mean = control_sum / 3

In [15]:
x = np.linspace(0, 36, 36)
names = ['Control', 'Control ensemble mean', 'EQ 60 Tg', 'EQ 15 Tg','45$^\circ$N 60 Tg','45$^\circ$N 15 Tg', '45$^\circ$S 60 Tg',  '45$^\circ$S 15 Tg']
letter_labels = ('a)', 'b)', 'c)', 'd)', 'e)', 'f)', 'g)', 'h)', 'i)','j)' ,'k)' ,'l)' )

In [ ]:
fig, axs = plt.subplots(4,2,sharey = True, sharex = True, figsize = (17,27))

ax = axs.ravel()

i = 0
for eruption in areaav_list[0:3]: 
    i +=2
    y = eruption.coord('pressure').points
    data = np.transpose(eruption.data)
    cf = ax[i].contourf(x,y,data, contour_levels, extend = 'both')
    ax[i].set_ylim(1,100)
    ax[i].xaxis.set_tick_params(width=2, labelsize = 25, direction = 'out', length = 6)
    ax[i].yaxis.set_tick_params(width=2, labelsize = 25, direction = 'out', length = 6)
    ax[i].set_ylabel('Pressure / hPa', fontsize = 30, labelpad = 18)
    ax[i].set_title(letter_labels[i], size=25, loc = 'left', y = 1.05)
    ax[i].set_title(names[i], size=30, y = 1.05)
    ax[i].set_yscale('log')
    ax[i].invert_yaxis()
    

data = np.transpose(areaav_list_cont[0].data)
cf = ax[0].contourf(x,y,data, contour_levels, extend = 'both')
ax[0].set_ylim(1,100)
ax[0].xaxis.set_tick_params(width=2, labelsize = 25, direction = 'out', length = 6)
ax[0].yaxis.set_tick_params(width=2, labelsize = 25, direction = 'out', length = 6)
ax[0].set_ylabel('Pressure / hPa', fontsize = 30, labelpad = 18)
ax[0].set_title(letter_labels[1], size=25, loc = 'left', y = 1.05)
ax[0].set_title(names[1], size=30, y = 1.05)
ax[0].set_yscale('log')
ax[0].invert_yaxis()
    
i = 1
for eruption in areaav_list[3:6]:
    i +=2
    y = eruption.coord('pressure').points
    data = np.transpose(eruption.data)
    cf = ax[i].contourf(x,y,data, contour_levels, extend = 'both')
    ax[i].xaxis.set_tick_params(width=2, labelsize = 25, direction = 'out', length = 6)
    ax[i].yaxis.set_tick_params(width=2, labelsize = 25, direction = 'out', length = 6)
    ax[i].set_title(letter_labels[i], size=25, loc = 'left', y = 1.05)
    ax[i].set_title(names[i], size=30, y = 1.05)
    ax[i].invert_yaxis()
    
data = np.transpose(control_mean.data)
cf = ax[1].contourf(x,y,data, contour_levels, extend = 'both')
ax[1].set_ylim(1,100)
ax[1].xaxis.set_tick_params(width=2, labelsize = 25, direction = 'out', length = 6)
ax[1].yaxis.set_tick_params(width=2, labelsize = 25, direction = 'out', length = 6)
ax[1].set_title(letter_labels[1], size=25, loc = 'left', y = 1.05)
ax[1].set_title(names[1], size=30, y = 1.05)
ax[1].set_yscale('log')
ax[1].invert_yaxis()

ax[6].set_xlabel('Months since start of simulation', fontsize = 30, labelpad = 20)    
ax[7].set_xlabel('Months since start of simulation', fontsize = 30, labelpad = 20)

plt.suptitle('January, e-QBO', size=40, y = 1.19)

cf = plt.contourf(x,y,np.transpose(control_av.data), contour_levels, extend = 'both')
time_vs_pressure_plot('Control')
plt.suptitle('QBO zonal wind Jan E', fontsize = 70)
plt.gca().invert_yaxis()

cbar_ax = fig.add_axes([0.8, 0.2, 0.04, 0.8])
colourbar = plt.colorbar(cf, cax = cbar_ax , orientation = 'vertical')
colourbar.set_label('m s', fontsize = 35)
colourbar.ax.tick_params(labelsize=30)

plt.tight_layout() 
plt.subplots_adjust( bottom = 0.2, wspace = 0.1, hspace = 0.2) 
plt.savefig('JNE_QBO.png')